# Music Sales Analysis

- Top Selling Artists
- Total Sales per Agent
- Sales by Country

### Connect the Jupyter Notebook to the database file

In [2]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

In [5]:
%%sql
SELECT name, type
FROM sqlite_master
WHERE type IN ('table','view');

 * sqlite:///chinook.db
   sqlite:///factbook.db
Done.


name,type
albums,table
sqlite_sequence,table
artists,table
customers,table
employees,table
genres,table
invoices,table
invoice_items,table
media_types,table
playlists,table


The database structure:

1. **Employees**: Employee ID, Last Name, First Name, ReportsTo
2. **Customers**: [List of customer data fields]
3. **Invoices**: Invoice ID, Date, Customer ID, [Other invoice-related fields]
4. **Invoice_Items**: [List of invoice line item data fields]
5. **Artists**: Artist ID, Name
6. **Albums**: Album ID, Artist ID, [Other album-related fields]
7. **Media_Types**: Media Type ID, Media Type Name
8. **Genres**: Genre ID, Genre Name
9. **Tracks**: Track ID, Album ID, Genre ID, Media Type ID, [Other track-related fields]
10. **Playlists**: Playlist ID, Playlist Name
11. **Playlist_Track**: Playlist ID, Track ID

### Top Selling Genres USA

In [80]:
%%sql

WITH usa_sold_tracks AS (
    SELECT il.* FROM invoice_Items il
        INNER JOIN invoices i on il.invoiceId = i.invoiceId
        INNER JOIN customers c on i.customerId = c.customerId
        WHERE c.country = "USA"
)


SELECT
    g.name genre,
    count(uts.invoiceLineId) tracks_sold,
    ROUND((cast(count(uts.InvoiceLineId) AS FLOAT) / (
        SELECT COUNT(*) from usa_sold_tracks
    ))*100,2) percentage_sold
FROM usa_sold_tracks uts
INNER JOIN tracks t on t.trackId = uts.trackId
INNER JOIN genres g on g.genreId = t.genreId
GROUP BY 1
ORDER BY 2 DESC

 * sqlite:///chinook.db
   sqlite:///factbook.db
Done.


genre,tracks_sold,percentage_sold
Rock,157,31.78
Latin,91,18.42
Metal,64,12.96
Alternative & Punk,50,10.12
Jazz,22,4.45
Blues,15,3.04
TV Shows,14,2.83
R&B/Soul,12,2.43
Comedy,8,1.62
Classical,8,1.62


### Total Sales by Agent

1. Write a query that finds the total dollar amount of sales assigned to each sales support agent within the company. Add any extra attributes for that employee that you find are relevant to the analysis.
2. Write a short statement describing your results, and providing a possible interpretation.

In [139]:
%%sql
SELECT 
    e.employeeID,
    e.title,
    e.firstName || ' ' || e.lastName as 'Name',
    COUNT(DISTINCT c.customerId) as 'Num of Customers',
    ROUND(SUM(i.total),2) as 'Total Sales',
    ROUND(SUM(i.total)/COUNT(DISTINCT c.customerId),2) as 'Dollar per Cust'
FROM employees AS e
LEFT JOIN customers AS c ON e.employeeId = c.supportRepId 
LEFT JOIN invoices as i ON i.customerId = c.customerId
GROUP BY e.employeeId;

 * sqlite:///chinook.db
   sqlite:///factbook.db
Done.


EmployeeId,Title,Name,Num of Customers,Total Sales,Dollar per Cust
1,General Manager,Andrew Adams,0,None,None
2,Sales Manager,Nancy Edwards,0,None,None
3,Sales Support Agent,Jane Peacock,21,833.04,39.67
4,Sales Support Agent,Margaret Park,20,775.4,38.77
5,Sales Support Agent,Steve Johnson,18,720.16,40.01
6,IT Manager,Michael Mitchell,0,None,None
7,IT Staff,Robert King,0,None,None
8,IT Staff,Laura Callahan,0,None,None


When looking at the breakdown of cost per customer it is evident that the difference between the Sales Support Agent is minimal. Based on the narrow range they all perform the same.

### Sales By Country

In [170]:
%%sql
SELECT 
    c.country,
    ROUND(SUM(i.total),2) as 'Total Sales',
    COUNT(DISTINCT c.customerID) as 'Number of Customers',
    ROUND(SUM(i.total)/COUNT(DISTINCT c.customerID),2) as 'Avg Sales per Customer'
FROM customers c, invoices i
WHERE c.customerID = i.customerID
GROUP BY c.Country
ORDER BY SUM(i.total) DESC
LIMIT 10

 

 * sqlite:///chinook.db
   sqlite:///factbook.db
Done.


Country,Total Sales,Number of Customers,Avg Sales per Customer
USA,523.06,13,40.24
Canada,303.96,8,38.0
France,195.1,5,39.02
Brazil,190.1,5,38.02
Germany,156.48,4,39.12
United Kingdom,112.86,3,37.62
Czech Republic,90.24,2,45.12
Portugal,77.24,2,38.62
India,75.26,2,37.63
Chile,46.62,1,46.62


The USA has the highest volume and number of sales. However average sales per customer is similar across all countries.